In [117]:
import os, sys, time, shutil
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None
import h5py

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [118]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'
survey_path = '../../../inputs/base_year/survey'

In [119]:
pd.options.display.float_format = '{:0,.0f}'.format

In [120]:
def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Congested network components by time of day
    df.columns

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    
    return df

In [121]:
# Create outputs dir 
dir = r'..\..\..\outputs\compare\RTP\standard'
if os.path.exists(dir):
    shutil.rmtree(dir)
os.makedirs(dir)

# Households and Population

In [122]:
# Total Population
# Load Network Summary sheet from network_summary_detailed
df = pd.read_csv(os.path.join(relative_path,'agg/dash',r'pptyp_county.csv'))
df = df.groupby('person_county').sum()[['psexpfac']]
df.loc['Total',:] = df.sum()
df = df.reset_index()
df.rename(columns={'psexpfac': 'Population', 'person_county': 'County'}, inplace=True)
df_hh = pd.read_csv(os.path.join(relative_path,'agg/dash',r'hh_geog.csv'))
df_hh = df_hh.groupby('hh_county').sum()[['hhexpfac']]
df_hh.loc['Total',:] = df_hh.sum()
df_hh = df_hh.reset_index()
df_hh.rename(columns={'hhexpfac': 'Households', 'hh_county': 'County'}, inplace=True)

df = df_hh.merge(df, on='County')
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','1_total_population.csv'),
         index=False)
HTML(df.to_html(index=False))

County,Households,Population
King,"884,650","2,149,240"
Kitsap,"101,859","257,159"
Pierce,"323,804","851,709"
Snohomish,"294,940","794,979"
Total,"1,605,253","4,053,087"


# Employment
By Workplace Location

In [123]:
df = pd.read_csv(os.path.join(relative_path,'agg/dash',r'person_worker_type.csv'))
df = df.groupby('person_work_county').sum()[['psexpfac']]
df.loc['Total',:] = df.sum()
df = df.reset_index()
df.rename(columns={'psexpfac': 'Population', 'person_work_county': 'County'}, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','2_total_employment.csv'))
HTML(df.to_html(index=False))

County,Population
Total,0


# Vehicle Miles Traveled
Includes truck and external trips

In [124]:
df_network = load_network_summary(os.path.join(relative_path,'network',r'network_results.csv'))
df = df_network.groupby('county').sum()[['VMT']]
df.loc['Total',:] = df.sum()
df = df.reset_index()
df.rename(columns={'county':'County'}, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','3_vmt.csv'))
HTML(df.to_html(index=False))

County,VMT
King,"47,596,321"
Kitsap,"4,673,940"
Pierce,"18,496,087"
Snohomish,"16,864,814"
Total,"87,631,161"


# Vehicle Hours Traveled

In [125]:
df = df_network.groupby('county').sum()[['VHT']]
df.loc['Total',:] = df.sum()
df = df.reset_index()
df.rename(columns={'county':'County'}, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','4_vht.csv'))
HTML(df.to_html(index=False))

County,VHT
King,"1,560,226"
Kitsap,"135,084"
Pierce,"578,931"
Snohomish,"506,990"
Total,"2,781,230"


# Vehicle Hours of Delay

In [126]:
df = df_network.groupby('county').sum()[['total_delay']]
df.rename(columns={'total_delay': 'Total Delay Hours'}, inplace=True)
df.loc['Total',:] = df.sum()
df = df.reset_index()
df.rename(columns={'county':'County'}, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','5_delay.csv'))
HTML(df.to_html(index=False))

County,Total Delay Hours
King,"260,453"
Kitsap,"6,723"
Pierce,"72,196"
Snohomish,"63,797"
Total,"403,169"


# Transit Boardings

In [127]:
df = pd.read_csv(os.path.join(relative_path,'transit',r'daily_boardings_by_agency.csv'), index_col='agency_name')
df.loc['Total',:] = df['boardings'].sum()
df = df.reset_index()
df.rename(columns={'agency_name':'Agency', 'boardings': 'Daily Boardings'}, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','6_transit_boardings.csv'))
HTML(df.to_html(index=False))

Agency,Daily Boardings
King County Metro,"399,607"
Sound Transit,"169,975"
Pierce Transit,"36,978"
Community Transit,"33,885"
Kitsap Transit,"15,106"
Washington Ferries,"12,643"
Everett Transit,"5,282"
Total,"673,477"


# Mode Share
All Trips

In [128]:
pd.options.display.float_format = '{:0,.1%}'.format
df_trip = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_county.csv')
df = df_trip.groupby('mode').sum()[['trexpfac']]/df_trip['trexpfac'].sum()
df = df.reset_index()
df.rename(columns={'mode': 'Mode', 'trexpfac': 'Mode Share'}, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','7_mode_share.csv'))
HTML(df.to_html(index=False))

Mode,Mode Share
Bike,1.4%
HOV2,22.9%
HOV3+,15.8%
SOV,41.5%
School Bus,2.0%
TNC,1.0%
Transit,2.9%
Walk,12.4%


# Emissions
Daily Tons

In [129]:
pd.options.display.float_format = '{:0,.0f}'.format
df = pd.read_csv(os.path.join(relative_path,'emissions',r'emissions_summary.csv'))
df = df.groupby('pollutant_name').sum()
df.rename(columns={'start_tons': 'Start', 'intrazonal_tons': 'Intrazonal', 'interzonal_tons': 'Interzonal',
                  'total_daily_tons': 'Total Daily'}, inplace=True)
df = df[df.index.isin(['CO','NOx','PM25 Total','PM10 Total','CO2 Equivalent'])]
df = df.applymap(lambda x: x if x > 100 else str(round(x,1)))
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','8_emissions.csv'))
df

,Start,Intrazonal,Interzonal,Total Daily
pollutant_name,,,,
CO,186,2.0,300,488
CO2 Equivalent,"2,568",213,"40,801","43,582"
NOx,10.6,0.2,68.7,79.6
PM10 Total,0.5,0.0,5.8,6.3
PM25 Total,0.4,0.0,1.8,2.3


# Lane Miles

In [130]:

import geopandas as gpd
pd.options.display.float_format = '{:0,.0f}'.format


In [131]:
gdf = gpd.read_file(r'..\..\..\inputs\scenario\networks\shapefiles\MD\md_edges.shp')

gdf['Lane Miles'] = gdf['length']*gdf['lanes']

In [132]:
facility_type_dict = {
    1: 'Interstate',
    2: 'Other Freeway',
    3: 'Expressway',
    4: 'Ramp',
    5: 'Principal Arterial',
    6: 'Minor Arterial',
    7: 'Major Collector',
    8: 'Minor Collector',
    9: 'Local',
    10: 'Busway',
    11: 'Non-Motor',
    12: 'Light Rail',
    13: 'Commuter Rail',
    15: 'Ferry',
    16: 'Passenger-Only Ferry',
    17: 'Centroid Connector',
    18: 'Facility Connector',
    19: 'HOV Only Freeway',
    20: 'HOV Only Ramp',
    98: 'Weave Links'
}

ul3_dict = {
    0: 'Rail/Walk/Ferry',
    1: 'Freeway',
    2: 'Expressway',
    3: 'Urban Arterial',
    4: 'One-way Arterial',
    5: 'Centroid Connector',
    6: 'Rural Arterial'
}

county_dict = {
    33: 'King',
    35: 'Kitsap',
    53: 'Pierce',
    61: 'Snohomish'
}

In [133]:
gdf['Facility Type'] = gdf['FacilityTy'].map(facility_type_dict)
df = gdf.groupby(['Facility Type','FacilityTy']).sum()[['Lane Miles']].sort_values('FacilityTy').reset_index()
df = df.drop('FacilityTy', axis=1)
df

,Facility Type,Lane Miles
0,Interstate,"1,427"
1,Other Freeway,749
2,Expressway,271
3,Ramp,517
4,Principal Arterial,"3,326"
5,Minor Arterial,"4,138"
6,Major Collector,"3,078"
7,Minor Collector,224
8,Local,345
9,Busway,22


In [134]:
gdf['Facility Group'] = gdf['ul3'].map(ul3_dict)
df = gdf.groupby(['Facility Group','ul3']).sum()[['Lane Miles']].sort_values('ul3').reset_index()
df = df.drop('ul3', axis=1)
df

,Facility Group,Lane Miles
0,Rail/Walk/Ferry,"1,684"
1,Freeway,"2,177"
2,Expressway,745
3,Urban Arterial,"7,093"
4,One-way Arterial,201
5,Centroid Connector,"10,267"
6,Rural Arterial,"3,418"


In [135]:
# Load link attributes and join
df = pd.read_csv(r'..\..\..\inputs\scenario\networks\extra_attributes\md_link_attributes.in\extra_links_2.txt',
                  delim_whitespace=True)
df['id'] = df['inode'].astype('str')+'-'+df['jnode'].astype('str')
df = gdf.merge(df, on='id', how='left')
df['County'] = df['@countyid'].map(county_dict)
df = df.groupby(['County','@countyid']).sum()[['Lane Miles']].sort_values('@countyid').reset_index()
df = df.drop('@countyid', axis=1)
df

,County,Lane Miles
0,King,"11,991"
1,Kitsap,"2,312"
2,Pierce,"6,161"
3,Snohomish,"5,034"
